In [68]:
using LinearAlgebra
using ForwardDiff
using Symbolics
using StaticArrays
using BenchmarkTools

In [2]:
function f(x)
    @show x
    @show zero(x)
    @assert x > zero(x)
    sin(x)/x
end

f (generic function with 1 method)

In [3]:
ForwardDiff.derivative(f, 1.0)

x = Dual{ForwardDiff.Tag{typeof(f), Float64}}(1.0,1.0)
zero(x) = Dual{ForwardDiff.Tag{typeof(f), Float64}}(0.0,0.0)


-0.30116867893975674

In [4]:
ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}}(1.0,1.0) > 0

true

In [5]:
module My

struct Foo{T}
    a::T
    b::T
end
    
end

Main.My

In [6]:
methods(My.Foo)

# 1 method for type constructor:
[1] Main.My.Foo(a::T, b::T) where T in Main.My at In[5]:4

In [7]:
methods(My.Foo{Int})

# 1 method for type constructor:
[1] Main.My.Foo{T}(a, b) where T in Main.My at In[5]:4

In [8]:
My.Foo(a::T, b::T) where T<:Real = My.Foo{T}(a, b)

In [9]:
methods(My.Foo)

# 2 methods for type constructor:
[1] Main.My.Foo(a::T, b::T) where T<:Real in Main at In[8]:1
[2] Main.My.Foo(a::T, b::T) where T in Main.My at In[5]:4

In [10]:
@which My.Foo(1im, 2im)

Main.My.Foo(a::T, b::T) where T in Main.My at In[5]:4

In [11]:
@which My.Foo(1, 2)

Main.My.Foo(a::T, b::T) where T<:Real in Main at In[8]:1

In [50]:
G(x, A) = exp(-dot(x, A, x)/2)

G (generic function with 1 method)

In [15]:
struct Bar{F, G}
    f::F
    ∇f::G
end

In [16]:
function Bar(f)
    ∇f(x, param) = ForwardDiff.gradient(x -> f(x, param), x)
    Bar(f, ∇f)
end

Bar

In [20]:
bar = Bar(G)

Bar{typeof(G), var"#∇f#2"{typeof(G)}}(G, var"#∇f#2"{typeof(G)}(G))

In [25]:
A = SA[
    2 -1
    -1 2
]

2×2 SMatrix{2, 2, Int64, 4} with indices SOneTo(2)×SOneTo(2):
  2  -1
 -1   2

In [26]:
bar.f(SA[1, 2], A)

0.049787068367863944

In [27]:
bar.∇f(SA[1, 2], A)

2-element SVector{2, Float64} with indices SOneTo(2):
  0.0
 -0.14936120510359183

In [51]:
@variables a[axes(A)...], x[axes(SA[1, 2])...]

2-element Vector{Symbolics.Arr{Num, N} where N}:
 a[SOneTo(2),SOneTo(2)]
 x[SOneTo(2)]

In [52]:
aa = collect(a)

2×2 SizedMatrix{2, 2, Num, 2, Matrix{Num}} with indices SOneTo(2)×SOneTo(2):
 a[1, 1]  a[1, 2]
 a[2, 1]  a[2, 2]

In [56]:
collect(aa) - aa

2×2 SMatrix{2, 2, Num, 4} with indices SOneTo(2)×SOneTo(2):
 0  0
 0  0

In [32]:
xx = collect(x)

2-element SizedVector{2, Num, Vector{Num}} with indices SOneTo(2):
 x[1]
 x[2]

In [102]:
_G = G(xx, aa) # |> expand |> simplify

exp((-1//2)*(x[1]*a[1, 1] + x[2]*a[2, 1])*x[1] - ((1//2)*(x[1]*a[1, 2] + x[2]*a[2, 2])*x[2]))

In [103]:
_∇G = Symbolics.gradient(_G, xx)

2-element SizedVector{2, Num, Vector{Num}} with indices SOneTo(2):
 (-*x[1]*a[1, 1] - ((1//2)*x[2]*a[1, 2]) - ((1//2)*x[2]*a[2, 1]))*exp((-1//2)*(x[1]*a[1, 1] + x[2]*a[2, 1])*x[1] - ((1//2)*(x[1]*a[1, 2] + x[2]*a[2, 2])*x[2]))
  ((-1//2)*x[1]*a[1, 2] - ((1//2)*x[1]*a[2, 1]) - (x[2]*a[2, 2]))*exp((-1//2)*(x[1]*a[1, 1] + x[2]*a[2, 1])*x[1] - ((1//2)*(x[1]*a[1, 2] + x[2]*a[2, 2])*x[2]))

In [96]:
_∇G / _G

2-element SizedVector{2, Num, Vector{Num}} with indices SOneTo(2):
 ((-1//2)*a[1, 2] - ((1//2)*a[2, 1]))*x[2] - (x[1]*a[1, 1])
 ((-1//2)*a[1, 2] - ((1//2)*a[2, 1]))*x[1] - (x[2]*a[2, 2])

In [104]:
∇G = build_function(_∇G, xx, vec(aa); expression=Val(false))[1]

RuntimeGeneratedFunction(#=in Symbolics=#, #=using Symbolics=#, :((ˍ₋arg1, ˍ₋arg2)->begin
          #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:282 =#
          #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:283 =#
          let var"x[1]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg1[1]), var"x[2]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg1[2]), var"a[1, 1]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg2[1]), var"a[2, 1]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg2[2]), var"a[1, 2]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg2[3]), var"a[2, 2]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg2[4])
              #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:375 =#
              (SymbolicUtils.Code.create_array)(typeof(ˍ₋arg1), nothing, Val{1}(), Val{(2,)}(), (*

In [97]:
∇G(SA[1, 2], A)

2-element SVector{2, Float64} with indices SOneTo(2):
  0.0
 -0.14936120510359183

In [106]:
function Bar(f, x, param)
    x_sym = collect(x)
    param_sym = collect(param)
    _f = simplify(expand(f(x_sym, param_sym)))
    _∇f = Symbolics.gradient(_f, x_sym)
    ff = build_function(_f, vec(x_sym), vec(param_sym); expression=Val(false))
    ∇f = build_function(_∇f, vec(x_sym), vec(param_sym); expression=Val(false))[1]
    Bar(ff, ∇f)
end

Bar

In [107]:
@variables x[1:2] a[1:2, 1:2]
car = Bar(G, x, a)

Bar{RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋arg1, :ˍ₋arg2), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", (0xe8e4a897, 0x0a819225, 0x068af023, 0xc2872ebe, 0xd95a665b)}, RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋arg1, :ˍ₋arg2), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", (0x65d79911, 0x5e34d083, 0xb302542c, 0xda9f6066, 0xf5f1eee2)}}(RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋arg1, :ˍ₋arg2), Symbolics.var"#_RGF_ModTag", Symbolics.var"#_RGF_ModTag", (0xe8e4a897, 0x0a819225, 0x068af023, 0xc2872ebe, 0xd95a665b)}(quote
    #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:282 =#
    #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:283 =#
    let var"x[1]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg1[1]), var"x[2]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ₋arg1[2]), var"a[1, 1]" = #= D:\.julia\packages\SymbolicUtils\Hwe4r\src\code.jl:169 =# @inbounds(ˍ

In [108]:
x = SA[1, 2]
A = SA[2 -1; -1 2]
car.f(x, A)

0.049787068367863944

In [109]:
car.∇f(x, A)

2-element SVector{2, Float64} with indices SOneTo(2):
  0.0
 -0.14936120510359183

In [110]:
@btime G($x, $A)
@btime $bar.f($x, $A)
@btime $car.f($x, $A)

  10.822 ns (0 allocations: 0 bytes)
  11.100 ns (0 allocations: 0 bytes)
  171.429 ns (0 allocations: 0 bytes)


0.049787068367863944

In [111]:
@btime $∇G($x, $A)
@btime $bar.∇f($x, $A)
@btime $car.∇f($x, $A)

  412.000 ns (0 allocations: 0 bytes)
  14.830 ns (0 allocations: 0 bytes)
  552.406 ns (0 allocations: 0 bytes)


2-element SVector{2, Float64} with indices SOneTo(2):
  0.0
 -0.14936120510359183

In [119]:
dG(x, A) = (A + A')*x/2*G(x, A)
#dG2(x, A) = A*x*G(x, A)
@btime $dG2($x, $A)

  12.500 ns (0 allocations: 0 bytes)


2-element SVector{2, Float64} with indices SOneTo(2):
 0.0
 0.14936120510359183

In [115]:
dG(xx, aa)

2-element SizedVector{2, Num, Vector{Num}} with indices SOneTo(2):
 (x[1]*a[1, 1] + (1//2)*(a[1, 2] + a[2, 1])*x[2])*exp((-1//2)*(x[1]*a[1, 1] + x[2]*a[2, 1])*x[1] - ((1//2)*(x[1]*a[1, 2] + x[2]*a[2, 2])*x[2]))
 (x[2]*a[2, 2] + (1//2)*(a[1, 2] + a[2, 1])*x[1])*exp((-1//2)*(x[1]*a[1, 1] + x[2]*a[2, 1])*x[1] - ((1//2)*(x[1]*a[1, 2] + x[2]*a[2, 2])*x[2]))